In [11]:
from game import DuelGame
from player import DummyPlayer, Aggressive, Defensive, Balanced, Opportunist, Healer, RandomBiased
from data_processor import Tracker
from dataset_repo import DatasetRepository
from essential_types import DataSample

from dotenv import load_dotenv
from typing import List
import json
import hashlib
import random
import math
import os

load_dotenv()

config_fields = [
    "APP_VERSION",
    "MAX_TURNS_PER_GAME",
    "SAMPLES_COUNT_PER_RUN",
    "DUMMY_PLAYER_POLICIES_DATA_DISTRIBUTION_IN_RUN",
    "AGGRESSIVE_OPPONENTS",
    "AGGRESSIVE_DISTRIBUTION",
    "Defensive_OPPONENTS",
    "Defensive_DISTRIBUTION",
    "Balanced_OPPONENTS",
    "Balanced_DISTRIBUTION",
    "Healer_OPPONENTS",
    "Healer_DISTRIBUTION",
    "Opportunist_OPPONENTS",
    "Opportunist_DISTRIBUTION",
    "RandomBiased_OPPONENTS",
    "RandomBiased_DISTRIBUTION",
    "AGGRESSIVE_EPSILON",
    "AGGRESSIVE_ATTACK_BIAS",
    "AGGRESSIVE_HEAL_THRESHOLD",
    "AGGRESSIVE_ATTACK_THREAT_THRESHOLD",
    "AGGRESSIVE_ATTACK_THREAT_HISTORY_LENGTH",
    "DEFENSIVE_EPSILON",
    "DEFENSIVE_ATTACK_PROB_OPP_HP_LOW",
    "DEFENSIVE_OPP_HP_THRESHOLD",
    "DEFENSIVE_HEAL_BIAS",
    "DEFENSIVE_DEFENSE_BIAS",
    "DEFENSIVE_ATTACK_THREAT_THRESHOLD",
    "DEFENSIVE_ATTACK_THREAT_HISTORY_LENGTH",
    "DEFENSIVE_ATTACK_START_TURN_THRESHOLD",
    "BALANCED_EPSILON",
    "BALANCED_DOMINATION_MARGIN",
    "BALANCED_DESPERATION_MARGIN",
    "HEALER_EPSILON",
    "HEALER_HEAL_THRESHOLD",
    "HEALER_HEAL_BIAS",
    "HEALER_ATTACK_PROB",
    "OPPORTUNIST_EPSILON",
    "OPPORTUNIST_DECISION_THRESHOLD",
    "OPPORTUNIST_BASE_TURN_NUMBER",
    "OPPORTUNIST_HEAL_THRESHOLD",
    "OPPORTUNIST_HEAL_BIAS",
    "RANDOM_BIASED_W_ATTACK",
    "RANDOM_BIASED_W_DEFENSE",
    "RANDOM_BIASED_W_DODGE",
    "RANDOM_BIASED_W_HEAL"
]

In [12]:
def play_game_and_return_samples(player_1, player_2, max_turns, seed) -> List[DataSample]:
    game = DuelGame(player_1, player_2, max_turns, random.Random(seed))
    tracker = Tracker(game)
    game.set_tracker(tracker)
    game.play_game()

    created_samples = tracker.get_samples()
    return created_samples

def start_run(
    source_type: str,  # 'env' or 'db'
    template_id: int = None,  # Required if source_type='db'
    label: str = None,
    description: str = None,
    run_label: str = None,
    run_note: str = None,
    seed: int = None
):
    """
    Start a dataset run with specified parameters.
    
    Parameters:
    -----------
    source_type : str
        'env' to load config from environment (.env)
        'db' to load existing config template from database
    template_id : int, optional
        Required when source_type='db', the ID of the config template to load
    label : str, optional
        Config label (for new templates from env)
    description : str, optional
        Config description (for new templates from env)
    run_label : str, optional
        Optional label for this run
    run_note : str, optional
        Optional note for this run
    seed : int, optional
        Random seed for this run. If None, auto-generates
    
    Returns:
    --------
    dict with run_id, template_id, seed, and other relevant info
    """
    # Validate parameters
    if source_type not in ['env', 'db']:
        raise ValueError("source_type must be 'env' or 'db'")
    
    if source_type == 'db' and template_id is None:
        raise ValueError("template_id is required when source_type='db'")
    
    # Instantiate a DatasetRepository class
    config = get_run_configuration()

    db_path = os.getenv('DATABASE_PATH')
    data_repo = DatasetRepository(db_path)
    
    print(f"Dataset Run Initialization - Loading from {source_type.upper()}")
    
    # ----------------------------
    # LOAD CONFIG FROM ENVIRONMENT
    # ----------------------------
    if source_type == 'env':
        # Hash the config deterministically
        config_json = json.dumps(config, sort_keys=True)
        config_hash = hashlib.sha256(config_json.encode()).hexdigest()
        
        # Check config duplication
        existing_template_id = data_repo.is_config_available_given_hash(config_hash)
        
        if existing_template_id == False:
            print(f"Existing config template found (ID={existing_template_id})")
            template_id = existing_template_id
        else:
            print("No existing config found. Creating new template.")
            
            # Create template object
            template = {
                "app_version": int(config['APP_VERSION']),
                "label": label or "",
                "description": description or "",
                "config_json": config_json,
            }
            
            template_id = data_repo.create_config_template(**template)
            print(f"Created new config template with ID: {template_id}")
    
    # --------------------------------
    # LOAD CONFIG TEMPLATE FROM DB
    # --------------------------------
    elif source_type == 'db':
        try:
            existing_config = data_repo.get_config_template(template_id)['config_json']
            if existing_config['APP_VERSION'] != config['APP_VERSION']:
                raise ValueError(
                    f"Template version mismatch. Template: {existing_config['APP_VERSION']}, "
                    f"Current: {config['APP_VERSION']}"
                )
            print(f"Loaded config template {template_id}")
        except Exception as e:
            raise ValueError(f"Failed to load template {template_id}: {str(e)}")
    
    # ----------------------------
    # CREATE DATASET RUN
    # ----------------------------
    # Auto-generate seed if not provided
    if seed is None:
        seed = random.randint(0, 2**32 - 1)
    
    run_id = data_repo.create_run(
        template_id=template_id,
        label=run_label or "",
        samples_count=0,
        note=run_note or "",
        seed=seed
    )
    
    print(
        f"Run created successfully "
        f"(run_id={run_id}, run_label={run_label or 'None'}, "
        f"template_id={template_id}, seed={seed})"
    )
    
    # Prepare run-specific configuration
    samples_count = math.ceil(config['SAMPLES_COUNT_PER_RUN'] * 1.05)
    
    # ----------------------------
    # RUN GAME SIMULATIONS
    # ----------------------------
    # Master RNG for this run (deterministic)
    rng = random.Random(seed)
    
    ARCHETYPE_CLASSES = {
        "Aggressive": Aggressive,
        "Defensive": Defensive,
        "Balanced": Balanced,
        "Healer": Healer,
        "Opportunist": Opportunist,
        "RandomBiased": RandomBiased,
    }
    
    ARCHETYPE_ORDER = [
        "Aggressive",
        "Defensive",
        "Balanced",
        "Healer",
        "Opportunist",
        "RandomBiased",
    ]
    
    total_target_samples = samples_count
    all_samples = []
    
    # Policy-level distribution
    policy_distribution = config["DUMMY_PLAYER_POLICIES_DATA_DISTRIBUTION_IN_RUN"]
    
    for policy_name, policy_ratio in zip(ARCHETYPE_ORDER, policy_distribution):
        policy_target = math.ceil(total_target_samples * policy_ratio)
        
        opponents = config[f"{policy_name.upper()}_OPPONENTS"]
        opponent_dist = config[f"{policy_name.upper()}_DISTRIBUTION"]
        
        for opponent_name, opp_ratio in zip(opponents, opponent_dist):
            pair_target = math.ceil(policy_target * opp_ratio)
            pair_collected = 0
            
            while pair_collected < pair_target:
                remaining = pair_target - pair_collected
                
                # Adjust max_turns only for final game if needed
                max_turns = min(
                    config["MAX_TURNS_PER_GAME"],
                    remaining
                )
                
                # Create players with shared RNG
                p1 = DummyPlayer(ARCHETYPE_CLASSES[policy_name], rng)
                p2 = DummyPlayer(ARCHETYPE_CLASSES[opponent_name], rng)
                
                samples = play_game_and_return_samples(
                    player_1=p1,
                    player_2=p2,
                    max_turns=max_turns,
                    seed=rng.randint(0, 2**32 - 1)
                )
                
                all_samples.extend(samples)
                pair_collected += len(samples)
    
    print(f"Total samples collected: {len(all_samples)}")
    
    # Store samples in database
    data_repo.store_samples(all_samples, run_id)
    
    # Return run information
    return {
        "run_id": run_id,
        "template_id": template_id,
        "seed": seed,
        "samples_collected": len(all_samples),
        "run_label": run_label,
        "run_note": run_note
    }
    

def get_run_configuration():
    config = {}

    for variable_name in config_fields:
        config[variable_name] = os.getenv(variable_name)

    return config

In [14]:
parameters = {
    "source_type" : "env",
    "template_id" : None,
    "label" : "Primary Version first Config Template",
    "description": "",
    "run_label": "first run",
    "run_note": None,
    "seed": None
}

start_run(**parameters)


Dataset Run Initialization - Loading from ENV
No existing config found. Creating new template.
Created new config template with ID: 1


TypeError: DatasetRepository.create_run() missing 1 required positional argument: 'samples_count'